# Preparation

<b>Libraries</b>

In [8]:
# bs4
import requests
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

# data structures
import numpy as np
import pandas as pd

# database
import sqlite3
import csv

# others
import sys, os, re, glob, datetime, string, time
from lxml import etree
from typing import Iterator

# API keys
cwd = os.getcwd()
os.chdir('../'); src_path = os.getcwd()
os.chdir(cwd)
if src_path not in sys.path:
    sys.path.append(src_path)
from src import redfin_config

<b>UDC</b>

In [9]:
# 
class RedfinHeadlessChromeBrowser():
    def __init__(
            self,
            default_download_dir: str, EMAIL: str, PASSWORD: str,
            user_agent: str=r'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36', 
            homepage_redfin_url: str=r'https://www.redfin.com/'
    ) -> None:
        self.default_download_dir = default_download_dir
        self.user_agent = user_agent
        self.homepage_redfin_url = homepage_redfin_url
        self.EMAIL = EMAIL; self.PASSWORD = PASSWORD

    # open website
    def _open_redfin_website(self):
        # 
        chrome_options = ChromeOptions()
        chrome_options.add_argument(f'user-agent={self.user_agent}')
        chrome_options.add_argument('--headless')
        #
        prefs = {"download.default_directory": self.default_download_dir, 
                 "download.directory_upgrade": True, 
                 "download.prompt_for_download": False}
        chrome_options.add_experimental_option('prefs', prefs)
        # 
        self.browser = webdriver.Chrome(options=chrome_options)
        self.browser.get(self.homepage_redfin_url)

        return self
    # log in to redfin account
    def _log_in_to_redfin_account(self) -> None:
        time.sleep(5)
        # begin logging
        self.browser.find_element(By.XPATH, "//span[text()='Join / Sign in']/..").click()
        
        # email
        time.sleep(1)
        self.browser.find_element(By.XPATH, "//input[@name='emailInput']").send_keys(self.EMAIL)
        self.browser.find_element(By.XPATH, "//span[text()='Continue with Email']/..").click()

        # password
        time.sleep(1)
        self.browser.find_element(By.XPATH, "//input[@name='passwordInput']").send_keys(self.PASSWORD)
        self.browser.find_element(By.XPATH, "//span[text()='Continue with Email']/..").click()

    # return brower
    def get_logged_in_browser(self) -> None:
        self._open_redfin_website()
        self._log_in_to_redfin_account()
    
    # quit browser
    def quit_browser(self) -> None:
        self.browser.quit()

    # return to homepage
    def return_to_homepage(self) -> None:
        self.browser.get(self.homepage_redfin_url)

In [28]:
# 
class CSVLoaderFromAPI():
    def __init__(self, paths: dict, redfin: RedfinHeadlessChromeBrowser) -> None:
        self.paths = paths
        self.redfin = redfin
    
    # create a table for each city
    def _create_table(self, table_name: str, column_names: list) -> None:
        with sqlite3.connect(f"{self.paths['target']}/api.db") as conn:
            cur = conn.cursor()
            
            # add scrapped time & 
            ## & prevent duplicated house and update new data if available
            cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name}(\
                                                        scrapped_time TEXT DEFAULT CURRENT_TIMESTAMP,\
                                                        address TEXT UNIQUE)")
            # existings columns names
            cur.execute(f"PRAGMA table_info({table_name})")
            existing_column_names = [ele[1] for ele in cur.fetchall()]
            # add default columns names
            for name in column_names:  
                if name in existing_column_names:
                    pass
                # let sqlite decide automatically
                else:
                    cur.execute(f"ALTER TABLE {table_name} ADD COLUMN {name}")
            conn.commit(); cur.close()
            
    # insert data into table 
    def _insert_data_to_table(self, table_name:str, row: dict) -> None:
        with sqlite3.connect(f"{self.paths['target']}/api.db") as conn:
            cur = conn.cursor()
            cur.execute(f"\
                        INSERT OR REPLACE INTO {table_name}{tuple(row.keys())} VALUES{tuple(row.values())}")
            conn.commit(); cur.close()
            
    # retrieve csv files
    def _retrive_link_from_db(self) -> Iterator[tuple[str, str]]:
        # retrive data from db
        with sqlite3.connect(f"{self.paths['target']}/urls.db") as conn:
            cur = conn.cursor()
            cur.execute("SELECT city, csv_download_link FROM urls \
                            WHERE csv_download_link IS NOT NULL")
            rows = cur.fetchall()
            conn.commit(); cur.close()
        
        # yield row to perform the ETL task iteratively
        if rows:
            for row in rows:
                yield row
        else:
            raise Exception('Can\'t retrive data from \"urls.db\"')
        
    # preprocess csv file
    def __adjust_column_name(self, text: str):
        # remove unrecoginized tokens
        text = text.lower().replace('#', '').replace('$', 'dollars').replace('/', "_per_")
        # remove white space
        text = re.sub(r"\s", '_', 
                        re.sub(r"\s\([^'']*\)", '', text))
        return text
    def _transform(self, city: str) -> Iterator[dict]:
        # read the csv file
        file_path = self.paths['temp'] + r"/" + os.listdir(self.paths['temp'])[0]
        with open(file_path) as f:
            reader = csv.reader(f, delimiter=',')
            rows = [row for i, row in enumerate(list(reader)) if i != 1]

            # create names of columns
            column_names = list(map(self.__adjust_column_name, rows.pop(0)))
            
            # convert them into format column_name:value for each row like CsvDictReader 
            data_to_insert = [{name:value for (name, value) in zip(column_names, row)} for row in rows]

        # clear the temp directory
        os.system(f"rm -rf {self.paths['temp']}/*")

        # create a table for a specific city
        self._create_table(city, column_names)

        # yiled data to insert into table (for memory efficiency)
        print(f'{city}\n{data_to_insert[:2]}')
        if data_to_insert:
            for data in data_to_insert:
                yield data
        else:
            raise Exception(f'The list with length of {len(data_to_insert)} is empty which mean the csv file is EMPTY for the city {city}!!!')

    # load data to dabase
    def _create_logs_table(self) -> None:
        with sqlite3.connect(f"{self.paths['target']}/api.db") as conn:
            cur = conn.cursor()

            # add logs table
            cur.execute("CREATE TABLE IF NOT EXISTS logs(\
                                                        city TEXT,\
                                                        csv_download_link TEXT,\
                                                        time TEXT DEFAULT CURRENT_TIMESTAMP)")
            conn.commit(); cur.close()
    def _insert_logs_data(self, city:str, csv_download_link: str) -> None:
        with sqlite3.connect(f"{self.paths['target']}/api.db") as conn:
            cur = conn.cursor()
            cur.execute("INSERT INTO logs(city, csv_download_link)\
                            VALUES(?, ?, ?)", (city, csv_download_link))

    def load(self):
        self._create_logs_table()
        # create each table iteratively
        for city, csv_download_link in self._retrive_link_from_db():
            self.redfin.browser.get(csv_download_link); time.sleep(7)
            
            # insert each row into table iteratively
            try:
                for row in self._transform(city):
                    self._insert_data_to_table(city, row)
            except:
                self._insert_logs_data(city, csv_download_link)
                # use "continue" keyword to next loop after catching an error
                continue

In [11]:
# 
class CSVLoaderFromScrapping():
    def __init__(self, paths: dict, redfin: RedfinHeadlessChromeBrowser) -> None:
        self.paths = paths
        self.redfin = redfin 

In [12]:
# 
class RedfinScrapper():
    def __init__(self, paths: dict, redfin: RedfinHeadlessChromeBrowser) -> None:
        self.paths = paths
        self.redfin = redfin

    def _create_table(self) -> None:
        with sqlite3.connect(f"{self.paths['target']}/urls.db") as conn:
            cur = conn.cursor()
            cur.executescript("CREATE TABLE IF NOT EXISTS urls(\
                                                            city TEXT UNIQUE, url TEXT, csv_download_link TEXT)")
            cur.close()

    def _get_city_urls(self) -> None:
        time.sleep(1)
        # get tags
        title = self.redfin.browser.find_element(By.XPATH, "//span[text()='Search for homes by city']")
        try:
            title.find_element(By.XPATH, "./parent::*/descendant::span[text()='Show more']").click()
        except:
            pass
        city_list = title.find_elements(By.XPATH, "./parent::*/descendant::li[@class='city']")

        # get names & links
        city = list()
        for li in city_list:
            tag = li.find_element(By.XPATH, ".//child::a")

            # make the name appproriate for sqlite3, includes: get rid of whitespaces, hyphens
            name = tag.text.lower().strip().replace(' real estate', '')\
                .strip().replace(' ', '_').replace('.', '_')
            link = tag.get_attribute('href')
            state_abbreviation = re.search(r"[A-Z]{2}", link).group()
            city.append([f"{name}_{state_abbreviation}", link])
        self.city = city
            
    # 
    def _get_available_csv_download_link(self) -> None:
        csv_download_links = list()
        for city_name, city_link in self.city:
            try:
                self.redfin.browser.get(city_link)
                time.sleep(1)
                csv_link = self.redfin.browser.find_element(By.XPATH, "//a[text()='(Download All)']")\
                    .get_attribute('href')
                csv_download_links.append([city_name, city_link, csv_link])
            except:
                csv_download_links.append([city_name, city_link, None])

        # 
        with sqlite3.connect(f"{self.paths['target']}/urls.db") as conn:
            cur = conn.cursor()
            for row in csv_download_links:
                cur.execute(f'\
                            INSERT OR REPLACE INTO urls VALUES(?, ?, ?)', tuple(row))
            cur.close()

    def start_scrapping(self) -> None:
        self._create_table()
        self._get_city_urls()
        self._get_available_csv_download_link()

# Selenium

<b>Draft</b>

In [6]:
with requests.Session() as s:
    # get respone of the GET request
    user_agent = r'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    headers = {'User-Agent': user_agent}
    test_url = 'https://www.redfin.com/city/30818/TX/Austin'
    r = s.get(test_url, headers=headers)

    # soup
    soup = BeautifulSoup(r.content)

    # DOM object
    dom = etree.HTML(str(soup))

In [21]:
map_home_cards = dom.xpath("//div[contains(@id, 'MapHomeCard')]")
eval(map_home_cards[0].xpath(".//descendant::script")[0].text)[0]

{'@context': 'http://schema.org',
 'name': '11516 Oltons Bluff Cv, Austin, TX 78754',
 'url': 'https://www.redfin.com/TX/Austin/11516-Oltons-Bluff-Cv-78754/home/31090991',
 'address': {'@type': 'PostalAddress',
  'streetAddress': '11516 Oltons Bluff Cv',
  'addressLocality': 'Austin',
  'addressRegion': 'TX',
  'postalCode': '78754',
  'addressCountry': 'US'},
 'geo': {'@type': 'GeoCoordinates',
  'latitude': 30.3748589,
  'longitude': -97.6482833},
 'numberOfRooms': 3,
 'floorSize': {'@type': 'QuantitativeValue', 'value': 2476, 'unitCode': 'FTK'},
 '@type': 'SingleFamilyResidence'}

In [12]:
tag_content = re.sub(r'\\', '', dom.xpath("//script")[-2].text)
text = re.findall(r"\"homes\":\[[^'']*\],\"dataSources\"", tag_content)
x1 = re.sub(r",\"dataSources\"",'', text[0])[8:]
x2 = re.sub(r"false", "False", x1)
x3 = re.sub(r"true", "True", x2)
eval(x3)[0]

{'mlsId': {'label': 'MLS#', 'value': '6723436'},
 'showMlsId': False,
 'mlsStatus': 'Active',
 'showDatasourceLogo': False,
 'price': {'value': 435000, 'level': 1},
 'hideSalePrice': False,
 'hoa': {'value': 20, 'level': 1},
 'isHoaFrequencyKnown': True,
 'sqFt': {'value': 2476, 'level': 1},
 'pricePerSqFt': {'value': 176, 'level': 1},
 'lotSize': {'value': 8132, 'level': 1},
 'beds': 3,
 'baths': 2.5,
 'fullBaths': 2,
 'partialBaths': 1,
 'location': {'value': 'Pioneer Crossing West Sec 07', 'level': 1},
 'stories': 2.0,
 'latLong': {'value': {'latitude': 30.3748589, 'longitude': -97.6482833},
  'level': 1},
 'streetLine': {'value': '11516 Oltons Bluff Cv', 'level': 1},
 'unitNumber': {'level': 1},
 'city': 'Austin',
 'state': 'TX',
 'zip': '78754',
 'postalCode': {'value': '78754', 'level': 1},
 'countryCode': 'US',
 'showAddressOnMap': True,
 'soldDate': 1184828400000,
 'searchStatus': 1,
 'propertyType': 6,
 'uiPropertyType': 1,
 'listingType': 1,
 'propertyId': 31090991,
 'listing

<b>Draft ver 01</b>

In [13]:
# prepare paths
cwd = os.getcwd()
os.chdir('../resource/data'); target = os.getcwd()
os.chdir('../temp'); temp = os.getcwd()
os.chdir(cwd)
paths = {'target': target, 'temp': temp}

In [14]:
# headless browser
redfin = RedfinHeadlessChromeBrowser(paths['temp'], redfin_config.EMAIL, redfin_config.PASSWORD)
redfin.get_logged_in_browser()

In [15]:
# redfin scrapper
scrapper = RedfinScrapper(paths, redfin)
scrapper.start_scrapping()

In [29]:
# load data from api
api = CSVLoaderFromAPI(paths, redfin)
api.load()

albuquerque_NM
[{'sale_type': 'MLS Listing', 'sold_date': '', 'property_type': 'Single Family Residential', 'address': '1809 Marble Ave NW', 'city': 'Albuquerque', 'state_or_province': 'NM', 'zip_or_postal_code': '87104', 'price': '849900', 'beds': '4', 'baths': '4.5', 'location': 'Albuquerque', 'square_feet': '4548', 'lot_size': '11325', 'year_built': '1936', 'days_on_market': '40', 'dollars_per_square_feet': '187', 'hoa_per_month': '', 'status': 'Active', 'next_open_house_start_time': '', 'next_open_house_end_time': '', 'url': 'https://www.redfin.com/NM/Albuquerque/1809-Marble-Ave-NW-87104/home/92012508', 'source': 'Southwest MLS', 'mls': '1068815', 'favorite': 'N', 'interested': 'Y', 'latitude': '35.0951712', 'longitude': '-106.6660328'}, {'sale_type': 'MLS Listing', 'sold_date': '', 'property_type': 'Single Family Residential', 'address': '10784 Pipestone Rd SW', 'city': 'Albuquerque', 'state_or_province': 'NM', 'zip_or_postal_code': '87121', 'price': '308500', 'beds': '3', 'baths'

OperationalError: table logs has no column named error

In [27]:
with sqlite3.connect(f"{paths['target']}/urls.db") as conn:
    cur = conn.cursor()
    # cur.execute('select city, csv_download_link from urls where csv_download_link IS NOT NULL')
    # rows = cur.fetchall()
    cur.execute("select * from urls where city='miami_FL'")
    rows = cur.fetchall()
    print(rows)
    conn.commit()
    cur.close()

[('miami_FL', 'https://www.redfin.com/city/11458/FL/Miami', 'https://www.redfin.com/stingray/api/gis-csv?al=3&has_att_fiber=false&has_deal=false&has_dishwasher=false&has_laundry_facility=false&has_laundry_hookups=false&has_parking=false&has_pool=false&has_short_term_lease=false&include_pending_homes=false&isRentals=false&is_furnished=false&is_income_restricted=false&is_senior_living=false&market=florida&num_homes=350&ord=redfin-recommended-asc&page_number=1&pool=false&region_id=11458&region_type=6&sf=1,2,3,5,6,7&status=9&travel_with_traffic=false&travel_within_region=false&uipt=1,2,3,4,5,6,7,8&utilities_included=false&v=8')]


In [19]:
with sqlite3.connect(f"{paths['target']}/api.db") as conn:
    cur = conn.cursor()
    cur.execute('select * from albuquerque_NM')
    rows = cur.fetchall()
    print(len(rows))
    conn.commit()
    cur.close()

349
